In [1]:
#! git clone https://github.com/seth814/Semantic-Shapes
! pip install pillow cython imgaug
! pip install git+https://github.com/lucasb-eyer/pydensecrf.git

You should consider upgrading via the '/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/bin/python -m pip install --upgrade pip' command.
  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-wnu_waj3
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/bin/python -m pip install --upgrade pip' command.


In [2]:
! git clone git@github.com:ElijahMingLiu/FCN-dataset.git

fatal: destination path 'FCN-dataset' already exists and is not an empty directory.


In [3]:
import numpy as np
import os
import cv2
import json
import tensorflow as tf

In [4]:
class DataGenerator(tf.keras.utils.Sequence):
    # Generates data for Keras
    def __init__(self, image_paths, annot_paths, batch_size=5,
                 shuffle=True, augment=False):
        # 图片路径
        self.image_paths = image_paths
        # annotation路径
        self.annot_paths = annot_paths
        # batch大小
        self.batch_size = batch_size
        # 是否在一个epoch之后打乱数据集
        self.shuffle = shuffle
        # 是否数据增强
        self.augment = augment
        # 执行一遍epoch结束后的操作
        # 这里是为了建立索引
        self.on_epoch_end()
    
    
    def __len__(self):
        # 每个epoch多少batch
        return int(np.floor(len(self.image_paths) / self.batch_size))
    
    
    def __getitem__(self, index):
        # 返回对应的第index个batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        image_paths = [self.image_paths[k] for k in indexes]
        annot_paths = [self.annot_paths[k] for k in indexes]
        
        # 根据路径生成numpy数据，y的格式为(batch_size,height, width, classes)
        # 相当于，第i个class都对应第i个channel
        X, y = self.__data_generation(image_paths, annot_paths)

        return X, y
    
    
    def on_epoch_end(self):
        # 每个epoch之后打乱索引
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    
    def get_poly(self, annot_path):
        # r阅读annotation文件
        with open(annot_path) as handle:
            data = json.load(handle)
        
        shape_dicts = data['shapes']

        return shape_dicts
    
    
    def create_multi_masks(self, im, shape_dicts):
        #每个channel是一个分类的mask
        channels = []
        # 根据上面的格式，可以看到label就是'star','rectangle等等'分类
        cls = [x['label'] for x in shape_dicts]
        # points就是组成这些的点的坐标
        poly = [np.array(x['points'], dtype=np.int32) for x in shape_dicts]
        # label2poly的格式类似
        # {'star': array([[ x1, y1],
        # [ x2,  y2],
        # ...)
        # 'square': array([[x1, y1],
        # [x2,  y2],
        # ...)
        # }
        label2poly = dict(zip(cls, poly))
        # 背景
        
        background = np.zeros(shape=(im.shape[0], im.shape[1]), dtype=np.float32)
        
        # 迭代跑每个object
        for i, label in enumerate(labels):
            # blank是object的mask
            blank = np.zeros(shape=(im.shape[0], im.shape[1]), dtype=np.float32)
            
            if label in cls:
                #cv2.fillPoly会根据points来画出对应的poly
                #分别填充blank和background
                cv2.fillPoly(blank, [label2poly[label]], 255)
                cv2.fillPoly(background, [label2poly[label]], 255)
            # 把blank加入channel
            channels.append(blank)

        # 如果有background对象，那么就也填充
        # 如果没有background对象，那么就翻转一下，有其他对象mask的地方为0，否则为255
        if 'background' in cls:
            background = np.zeros(shape=(im.shape[0], im.shape[1]), dtype=np.float32)
            cv2.fillPoly(background, [label2poly['background']], 255)
        else:
            _, background = cv2.threshold(background, 127, 255, cv2.THRESH_BINARY_INV)
        #把background也加入channel
        channels.append(background)
        # Y为[height, width, n_classes]，然后再除上255，转换为[0,1]区间
        Y = np.stack(channels, axis=2) / 255.0

        return Y
    
    def __data_generation(self, image_paths, annot_paths):
        
        # 建立训练用的numpy数组
        X = np.empty((self.batch_size, imshape[0], imshape[1], imshape[2]), dtype=np.float32)
        Y = np.empty((self.batch_size, imshape[0], imshape[1], n_classes),  dtype=np.float32)
        
        for i, (im_path, annot_path) in enumerate(zip(image_paths, annot_paths)):
            
            # 如果图片
            if imshape[2] == 1:
                im = cv2.imread(im_path, 0)
                im = np.expand_dims(im, axis=2)
            elif imshape[2] == 3:
                im = cv2.imread(im_path, 1)
                im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            shape_dicts = self.get_poly(annot_path)
            
            # check for augmentation
            if self.augment:
                im, shape_dicts = self.augment_poly(im, shape_dicts)
            
            # create target masks
            if n_classes == 1:
                mask = self.create_binary_masks(im, shape_dicts)
            elif n_classes > 1:
                mask = self.create_multi_masks(im, shape_dicts)
            
            X[i,] = im
            Y[i,] = mask
            
        return X, Y


In [5]:
def sorted_fns(dir):
    return sorted(os.listdir(dir), key=lambda x: int(x.split('.')[0]))


$$
intersection=y_{true}\cdot y_{pred}
$$

$$
\frac{2\times intersection+smooth}{\sum{y_{true}}+\sum{y_{pred}}+smooth}
$$

拆开fcn model

In [6]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, concatenate, Dropout,\
                                    Lambda, Conv2DTranspose, Add
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf
import os


def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x




In [8]:
def dice(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [7]:
# 模型名称
model_name = 'fcn_8'
# 是否加载预训练的模型
pretrained=False

# 图片尺寸设置
imshape = (256, 256, 3)

# 色域
hues = {'star': 30,
        'square': 0,
        'circle': 90,
        'triangle': 60}

# 标签
labels = sorted(hues.keys())

#分类数
n_classes = len(labels) + 1

#图片和标注的路径
image_paths = [os.path.join('FCN-dataset/images', x) for x in sorted_fns('images')]
annot_paths = [os.path.join('FCN-dataset/annotated', x) for x in sorted_fns('annotated')]

In [9]:
b = 4
i = Input(shape=imshape)
s = Lambda(lambda x: preprocess_input(x)) (i)
## Block 1
x = Conv2D(2**b, (3, 3), activation='elu', padding='same', name='block1_conv1')(s)
x = Conv2D(2**b, (3, 3), activation='elu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
f1 = x
# <tf.Tensor 'block1_pool/MaxPool_3:0' shape=(None, 128, 128, 16) dtype=float32>

# Block 2
x = Conv2D(2**(b+1), (3, 3), activation='elu', padding='same', name='block2_conv1')(x)
x = Conv2D(2**(b+1), (3, 3), activation='elu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
f2 = x
# <tf.Tensor 'block2_pool/MaxPool_3:0' shape=(None, 64, 64, 32) dtype=float32>

# Block 3
x = Conv2D(2**(b+2), (3, 3), activation='elu', padding='same', name='block3_conv1')(x)
x = Conv2D(2**(b+2), (3, 3), activation='elu', padding='same', name='block3_conv2')(x)
x = Conv2D(2**(b+2), (3, 3), activation='elu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
pool3 = x
# <tf.Tensor 'block3_pool/MaxPool_3:0' shape=(None, 32, 32, 64) dtype=float32>

# Block 4
x = Conv2D(2**(b+3), (3, 3), activation='elu', padding='same', name='block4_conv1')(x)
x = Conv2D(2**(b+3), (3, 3), activation='elu', padding='same', name='block4_conv2')(x)
x = Conv2D(2**(b+3), (3, 3), activation='elu', padding='same', name='block4_conv3')(x)
pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
# <tf.Tensor 'block4_pool/MaxPool_3:0' shape=(None, 16, 16, 128) dtype=float32>

# Block 5
x = Conv2D(2**(b+3), (3, 3), activation='elu', padding='same', name='block5_conv1')(pool4)
x = Conv2D(2**(b+3), (3, 3), activation='elu', padding='same', name='block5_conv2')(x)
x = Conv2D(2**(b+3), (3, 3), activation='elu', padding='same', name='block5_conv3')(x)
pool5 = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
# <tf.Tensor 'block5_pool/MaxPool_3:0' shape=(None, 8, 8, 128) dtype=float32>

# 和pool5
conv6 = Conv2D(2048 , (7, 7) , activation='elu' , padding='same', name="conv6")(pool5)
conv6 = Dropout(0.5)(conv6)
conv7 = Conv2D(2048 , (1, 1) , activation='elu' , padding='same', name="conv7")(conv6)
conv7 = Dropout(0.5)(conv7)
# <tf.Tensor 'dropout_7/cond/Identity:0' shape=(None, 8, 8, 2048) dtype=float32>

# pool4的卷积 16*16
pool4_n = Conv2D(n_classes, (1, 1), activation='elu', padding='same')(pool4)
# <tf.Tensor 'conv2d_6/Elu:0' shape=(None, 16, 16, 5) dtype=float32>

# # conv7 upsampling成16*16*5 + poo4
u2 = Conv2DTranspose(n_classes, kernel_size=(2, 2), strides=(2, 2), padding='same')(conv7)
u2_skip = Add()([pool4_n, u2])
# <tf.Tensor 'add_6/add:0' shape=(None, 16, 16, 5) dtype=float32>


pool3_n = Conv2D(n_classes, (1, 1), activation='elu', padding='same')(pool3)

# conv7 + pool4 upsampling + pool3
u4 = Conv2DTranspose(n_classes, kernel_size=(2, 2), strides=(2, 2), padding='same')(u2_skip)
u4_skip = Add()([pool3_n, u4])
# <tf.Tensor 'add_7/add:0' shape=(None, 32, 32, 5) dtype=float32>

# upsampling成256*256
o = Conv2DTranspose(n_classes, kernel_size=(8, 8), strides=(8, 8), padding='same',
                    activation='softmax')(u4_skip)


model = Model(inputs=i, outputs=o, name='fcn_multi')
model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=[dice])
# model.summary()

In [19]:
u4_skip

<tf.Tensor 'add_1/add:0' shape=(None, 32, 32, 5) dtype=float32>

In [12]:
o

<tf.Tensor 'conv2d_transpose_2/truediv:0' shape=(None, 256, 256, 5) dtype=float32>

In [10]:
dataGenerator = DataGenerator(image_paths, annot_paths)
model.fit_generator(generator=dataGenerator,
                    steps_per_epoch=len(dataGenerator),
                    epochs=500, verbose=1,
                    )


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/500
40/40 [==============================] - 21s 514ms/step - loss: 1.5777 - dice: 0.2123
Epoch 2/500
40/40 [==============================] - 21s 514ms/step - loss: 1.4580 - dice: 0.2675
Epoch 3/500
40/40 [==============================] - 21s 513ms/step - loss: 1.2873 - dice: 0.3513
Epoch 4/500
40/40 [==============================] - 21s 513ms/step - loss: 1.1126 - dice: 0.4394
Epoch 5/500
40/40 [==============================] - 21s 514ms/step - loss: 0.9529 - dice: 0.5251
Epoch 6/500
40/40 [==============================] - 21s 515ms/step - loss: 0.8187 - dice: 0.5929
Epoch 7/500
40/40 [==============================] - 22s 545ms/step - loss: 0.7105 - dice: 0.6559
Epoch 8/500
40/40 [==============================] - 21s 513ms/step - loss: 0.6354 - dice: 0.7024
Epoch 9/500
40/40 [==============================] - 21s 514ms/step - loss: 0.5726 - dice: 0.7339
Epoch 10/500
40/40 [======================

KeyboardInterrupt: 